## Data

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## Simple Tokenization

In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# lets tokenize the entire input dataset into a Torch Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Split into train/validation split | 90% train, 10% val
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Block Size

In [10]:
# train in blocks of 8
block_size = 8
train_data[:block_size+1] # 8 characters, plus the target character

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size] # first 8 characters
y = train_data[1:block_size+1] # targets are the next character in the sequence
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


## Batch Size
of training examples used in each iteration of a forward/backward pass.

batch_size = 4, block_size = 8 | 4 sets of length 8 training examples to be processed

In [12]:
torch.manual_seed(1337) # for reproducibility
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the max context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,)) # batch_size num. of random offsets in dataset
    x = torch.stack([data[i:i + block_size] for i in idx]) # (batch_size, block_size)
    y = torch.stack([data[i + 1:i + block_size + 1] for i in idx]) # (batch_size, block_size)
    return x, y # (batch_size, block_size), (batch_size, block_size)

xb, yb = get_batch('train')
print("INPUTS:")
print(xb.shape)
print(xb)
print("TARGETS:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t + 1] # input (b,t)
        target = yb[b, t] # target (b)
        print(f"when input is {context.tolist()} the target is {target}")

INPUTS:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
TARGETS:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target i

In [13]:
# out input to the transformer
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## Simple Bigram Language Model

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # for reproducibility

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # 1) Predictions: idx and targets are both (B, T) tesnor of integers
        logits = self.token_embedding_table(idx) # (batch, time, vocab_size)

        # 2) Evaluate loss
        if targets is None:
            loss = None
        else:
            # need to reshape to fix Pytorch's expectation of the shape of the logits
            B, T, C = logits.shape
            logits = logits.view(B * T, C) 
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets) # we have the identity of the next character, how well are the logits predicting it?
        
        return logits, loss
    
    """
    For each sequence in the batch, we will predict and concatenate the next character 
    along the Time dimension until max_new_tokens.
    """
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # 1) get the predictions
            logits, loss = self(idx)

            # 2) focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            # 3) apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # 4) sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # 5) append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) # (B, T) | starting newline character (0)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
# Create the PyTorch optimizer | gradient optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
# Create the training loop
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zero out the gradients
    loss.backward() # backpropagation
    optimizer.step() # update the weights

print(loss.item())

2.5727508068084717


In [17]:
# Sample from the model
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


## Self-Attention

In [18]:
# consider the following toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 #batch, time, channels
x = torch.randn(B, T, C) # (B, T, C)
print(x.shape)

torch.Size([4, 8, 2])


In [19]:
# IDEA: We want to get the mean of all previous time steps for each batch at each time step
# We want x[b, t] = mean_{i<=t} x[b, i] | mean of all previous time steps
# We can do this with a for loop, but it is slow
xbow = torch.zeros(B, T, C) # (B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1] # (t+1, C)
        xbow[b, t] = torch.mean(xprev, dim=0)

xbow[0] # Looking at the first batch

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [20]:
# We can leverage the lower triangular matrix to perform a matrix multiplication with a normalized matrix A that essentially does the same thing
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # lower triangular matrix
a = a / torch.sum(a, 1, keepdim=True) # normalize each row to sum to 1

b = torch.randint(0, 10, (3, 2)).float()

c = a @ b # matrix multiplication

print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [21]:
# Version 2: Using matrix multiply for weighted aggregation
wei = torch.tril(torch.ones(T, T)) # (T, T)
wei = wei / wei.sum(1, keepdim=True) # (T, T)

xbow2 = wei @ x # (T, T) @ (B, T, C) = (B, T, C)
torch.allclose(xbow, xbow2, rtol= 1e-04)

True

In [22]:
# Version 3: Softmax
wei = torch.zeros((T, T))
wei = wei.masked_fill(torch.triu(torch.ones(T, T), diagonal=1).bool(), float('-inf')) # -inf on the upper triangular matrix
wei = F.softmax(wei, dim=-1) # softmax to get the weights
xbow3 = wei @ x
torch.allclose(xbow, xbow3, rtol= 1e-04) # check if they are the same
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
# Version 4: Single Head Self-Attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C) # (B, T, C)

# lets see a single Head perform self-attention
# can think of each token as an independent identity.
# key - heres what I am
# query - heres what I am looking for
# value - if you find me interesting, heres what I will communicate to you
head_size = 16 # number of channels per head
key = nn.Linear(C, head_size, bias=False) # (C, head_size) - i.e. contains information about itself: the token
query = nn.Linear(C, head_size, bias=False) # (C, head_size) - i.e. contains information about what it is looking for
value = nn.Linear(C, head_size, bias=False) 
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) = (B, T, T)

tril = torch.tril(torch.ones(T, T)) # lower triangular matrix
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # -inf on the upper triangular matrix | don't allow future tokens to be seen
wei = F.softmax(wei, dim=-1) # softmax to get the weights

v = value(x) # (B, T, 16)
out =  wei @ v
out.shape

torch.Size([4, 8, 32])

Notes:
- Attention is a **communication mechanism** - can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that points to them, with data-dependent weights
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- "self-attention" just means that the keys and values are produced from the same source as queries, in "cross-attention" quries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module) - a separate entity that we might want to pool from
- "Scaled" attention additionally divides `wei` by `1/sqrt(head_size)`. This makes it so when input Q,K are unit variance, `wei` will be unit variance too and Softmax will stay diffuse and not saturate too much

In [25]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089